In [ ]:
def calc_MI_and_MaxL_fixed(data):
    total_downloads = data[2].sum()
    
    size_downloads = defaultdict(int)
    for row in data.itertuples(index=False):
        size_downloads[row[1]] += row[2]
        
    MI = 0
    
    for size, count in size_downloads.items():
        MI -= (count / total_downloads) * math.log2(count / total_downloads)
        
    MaxL = math.log2(len(size_downloads))
        
    return MI, MaxL

In [ ]:
%%cython --compile-args=-fopenmp --link-args=-fopenmp --force -a
cimport cython
cimport numpy as np
import numpy as np
from cython.parallel import prange
from collections import defaultdict


cdef extern from "math.h":
    double log2(double x) nogil

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef void init_p_ji(int i, int[:] i_max, int[:] cndl_idxs, double[:] p_ji, double[:] p_i) nogil:
    cdef int j, i_num_cndls, i_idx, offset
    cdef double block_prob
    
    block_prob = 0.
        
    for j in range(i, i_max[i]+1):
        block_prob += p_i[j]
        
    i_num_cndls = i_max[i] - i + 1
    i_idx = cndl_idxs[i]
        
    for offset in range(i_num_cndls):
        p_ji[i_idx + offset] = p_i[i + offset] / block_prob
    
    
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef void update_p_j(int j, long[:] j_min, int[:] cndl_idxs, double[:] p_i, double[:] p_ji, double[:] p_j) nogil:
    cdef double j_prob = 0.
    cdef int i
    
    for i in range(j_min[j], j+1):
        j_prob += p_i[i] * p_ji[cndl_idxs[i] + (j-i)]
            
    p_j[j] = j_prob    
            
            
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef double calc_p_j(int j, double[:] p_j) nogil:
    cdef double prob = p_j[j]
    
    if prob == 0.:
        return 0.
    else:
        return -prob * log2(prob)
            
    
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef double calc_p_ji(int i, int[:] i_max, int[:] cndl_idxs, double[:] p_ji, double[:] p_i) nogil:
    cdef double log_sum = 0.
    cdef double prob
    cdef int i_idx, idx, i_num_cndls, offset
        
    i_num_cndls = i_max[i] - i + 1
    i_idx = cndl_idxs[i]
        
    for offset in range(i_num_cndls):
        idx = i_idx + offset
        prob = p_ji[idx]
        if prob > 0.:
            log_sum += prob * log2(prob)
        
    return p_i[i] * log_sum


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef double calc_max_L(int j, long[:] j_min, int[:] cndl_idxs, double[:] p_ji) nogil:
    cdef double largest_cndl = 0.
    cdef double cndl
    cdef int i
    
    for i in range(j_min[j], j+1):
        cndl = p_ji[cndl_idxs[i] + (j-i)]
        if cndl > largest_cndl:
            largest_cndl = cndl
            
    return largest_cndl 


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def p_alpaca(data, double c):
    data_sorted = data.groupby(1, as_index=False).agg({2:"sum"})
    
    cdef double total_views = 1. / float(data_sorted[2].sum())
    data_sorted[3] = data_sorted[2] * total_views
    cdef int num_sizes = data_sorted.shape[0]
    
    cdef long[::1] s = data_sorted[1].to_numpy()
    cdef double[::1] p_i = data_sorted[3].to_numpy()
    
    cdef int[::1] i_max = np.empty(num_sizes, dtype=np.intc)
    cdef long[::1] j_min = np.full(num_sizes, num_sizes)
    
    cdef int[::1] cndl_idxs = np.empty(num_sizes, dtype=np.intc)
    
    cdef int i, j
    cdef double max_size
    cdef int num_cndls = 0
    
    for i in range(num_sizes):        
        max_size = float(s[i]) * c
        
        cndl_idxs[i] = num_cndls
        
        for j in range(i,num_sizes):
            if float(s[j]) > max_size:
                break
            i_max[i] = j
            num_cndls += 1
            if i < j_min[j]:
                j_min[j] = i
                
    cdef double[::1] p_ji = np.empty(num_cndls, dtype=np.double)
    cdef double[::1] p_j  = np.empty(num_sizes, dtype=np.double)
    
    for i in prange(num_sizes, nogil=True):
        init_p_ji(i, i_max, cndl_idxs, p_ji, p_i)
    
    for j in prange(num_sizes, nogil=True):
        update_p_j(j, j_min, cndl_idxs, p_i, p_ji, p_j)
    
    cdef double p_j_result = 0.
    cdef double p_ji_result = 0.
    cdef double MI
    
    for j in prange(num_sizes, nogil=True):
        p_j_result += calc_p_j(j, p_j)
        
    for i in prange(num_sizes, nogil=True):
        p_ji_result += calc_p_ji(i, i_max, cndl_idxs, p_ji, p_i)
        
    MI = p_j_result + p_ji_result
    
    cdef double max_L = 0.
    
    for j in prange(num_sizes, nogil=True):
        max_L += calc_max_L(j, j_min, cndl_idxs, p_ji)
        
    return (MI, max_L)

In [ ]:
%%cython --compile-args=-fopenmp --link-args=-fopenmp --force -a
cimport cython
cimport numpy as np
import numpy as np
from cython.parallel import prange
from collections import defaultdict

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef void init_p_ji(int i, int[:] i_max, int[:] cndl_idxs, double[:] p_ji, double[:] p_i) nogil:
    cdef int j, i_num_cndls, i_idx, offset
    cdef double block_prob
    
    block_prob = 0.
        
    for j in range(i, i_max[i]+1):
        block_prob += p_i[j]
        
    i_num_cndls = i_max[i] - i + 1
    i_idx = cndl_idxs[i]
        
    for offset in range(i_num_cndls):
        p_ji[i_idx + offset] = p_i[i + offset] / block_prob


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def p_alpaca_no_metrics(data, double c):
    data_sorted = data.groupby(1, as_index=False).agg({2:"sum"})
    
    cdef double total_views = 1. / float(data_sorted[2].sum())
    data_sorted[3] = data_sorted[2] * total_views
    cdef int num_sizes = data_sorted.shape[0]
    
    cdef long[::1] s = data_sorted[1].to_numpy()
    cdef double[::1] p_i = data_sorted[3].to_numpy()
    
    cdef int[::1] i_max = np.empty(num_sizes, dtype=np.intc)
    
    cdef int[::1] cndl_idxs = np.empty(num_sizes, dtype=np.intc)
    
    cdef int i, j
    cdef double max_size
    cdef int num_cndls = 0
    
    for i in range(num_sizes):        
        max_size = float(s[i]) * c
        
        cndl_idxs[i] = num_cndls
        
        for j in range(i,num_sizes):
            if float(s[j]) > max_size:
                break
            i_max[i] = j
            num_cndls += 1
                
    cdef double[::1] p_ji = np.empty(num_cndls, dtype=np.double)

    for i in prange(num_sizes, nogil=True):
        init_p_ji(i, i_max, cndl_idxs, p_ji, p_i)
        
    return p_ji


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def p_alpaca_no_metrics_return_vuln(data, double c, vuln_list):
    data_sorted = data.groupby(1, as_index=False).agg({2:"sum"})
    
    cdef double total_views = 1. / float(data_sorted[2].sum())
    data_sorted[3] = data_sorted[2] * total_views
    cdef int num_sizes = data_sorted.shape[0]
    
    cdef long[::1] s = data_sorted[1].to_numpy()
    cdef double[::1] p_i = data_sorted[3].to_numpy()
    
    cdef int[::1] i_max = np.empty(num_sizes, dtype=np.intc)
    
    cdef int[::1] cndl_idxs = np.empty(num_sizes, dtype=np.intc)
    
    cdef int i, j
    cdef double max_size
    cdef int num_cndls = 0
    
    for i in range(num_sizes):        
        max_size = float(s[i]) * c
        
        cndl_idxs[i] = num_cndls
        
        for j in range(i,num_sizes):
            if float(s[j]) > max_size:
                break
            i_max[i] = j
            num_cndls += 1
                
    cdef double[::1] p_ji = np.empty(num_cndls, dtype=np.double)

    for i in prange(num_sizes, nogil=True):
        init_p_ji(i, i_max, cndl_idxs, p_ji, p_i)
        
    size_to_i = {}
    for i in range(num_sizes):
        size_to_i[s[i]] = i
        
    size_is_vuln = defaultdict(int)
    size_no_vuln = defaultdict(int)
    size_counts = defaultdict(int)
    
    cdef long j_size
    cdef double prob
    
    for row in data.itertuples(index=False):
        i = size_to_i[row[1]]        
        i_num_cndls = i_max[i] - i + 1  
        i_idx = cndl_idxs[i]  
        
        if row[0] in vuln_list:
            for offset in range(i_num_cndls):
                j_size = s[i + offset]
                prob = p_ji[i_idx + offset]
                size_is_vuln[j_size] += prob*row[2]
                size_counts[j_size] += prob*row[2]
        else:
            for offset in range(i_num_cndls):
                j_size = s[i + offset]
                prob = p_ji[i_idx + offset]
                size_no_vuln[j_size] += prob*row[2]
                size_counts[j_size] += prob*row[2]
        
    return size_is_vuln, size_no_vuln, size_counts

In [ ]:
%%cython --compile-args=-fopenmp --link-args=-fopenmp --force -a
cimport cython
cimport numpy as np
import numpy as np
import pandas as pd
from cython.parallel import prange
from collections import defaultdict

cdef extern from "math.h":
    double log2(double x) nogil

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef void update_p_j(int j, long[:] j_min, int[:] cndl_idxs, double[:] p_i, double[:] p_ji, double[:] p_j) nogil:
    cdef double j_prob = 0.
    cdef int i
    
    for i in range(j_min[j], j+1):
        j_prob += p_i[i] * p_ji[cndl_idxs[i] + (j-i)]
            
    p_j[j] = j_prob
    
    
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef void update_p_ji(int i, int[:] i_max, int[:] cndl_idxs, double[:] p_ji, double[:] p_j) nogil:
    cdef double prob_sum = 0.
    cdef int j, i_idx, i_num_cndls, offset
        
    for j in range(i, i_max[i]+1):
        prob_sum += p_j[j]
            
    i_num_cndls = i_max[i] - i + 1
    i_idx = cndl_idxs[i]
            
    for offset in range(i_num_cndls):
        p_ji[i_idx+offset] = p_j[i+offset] / prob_sum
            
            
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef double calc_p_j(int j, double[:] p_j) nogil:
    cdef double prob = p_j[j]
    
    if prob == 0.:
        return 0.
    else:
        return -prob * log2(prob)
            
    
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef double calc_p_ji(int i, int[:] i_max, int[:] cndl_idxs, double[:] p_ji, double[:] p_i) nogil:
    cdef double log_sum = 0.
    cdef double prob
    cdef int i_idx, idx, i_num_cndls, offset
        
    i_num_cndls = i_max[i] - i + 1
    i_idx = cndl_idxs[i]
        
    for offset in range(i_num_cndls):
        idx = i_idx + offset
        prob = p_ji[idx]
        if prob > 0.:
            log_sum += prob * log2(prob)
        
    return p_i[i] * log_sum


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef double calc_max_L(int j, long[:] j_min, int[:] cndl_idxs, double[:] p_ji) nogil:
    cdef double largest_cndl = 0.
    cdef double cndl
    cdef int i
    
    for i in range(j_min[j], j+1):
        cndl = p_ji[cndl_idxs[i] + (j-i)]
        if cndl > largest_cndl:
            largest_cndl = cndl
            
    return largest_cndl 


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def BlahutArimoto(data, double c, long max_itr = 40, double eps = 1e-4):
    data_sorted = data.groupby(1, as_index=False).agg({2:"sum"})
    
    cdef double total_views = 1. / float(data_sorted[2].sum())
    data_sorted[3] = data_sorted[2] * total_views
    cdef int num_sizes = data_sorted.shape[0]
    
    cdef long[::1] s = data_sorted[1].to_numpy()
    cdef double[::1] p_i = data_sorted[3].to_numpy()
    
    cdef int[::1] i_max = np.empty(num_sizes, dtype=np.intc)
    cdef long[::1] j_min = np.full(num_sizes, num_sizes)
    
    cdef int[::1] cndl_idxs = np.empty(num_sizes, dtype=np.intc)
    
    cdef int i, j
    cdef double max_size
    cdef int num_cndls = 0
    
    for i in range(num_sizes):        
        max_size = float(s[i]) * c
        
        cndl_idxs[i] = num_cndls
        
        for j in range(i,num_sizes):
            if float(s[j]) > max_size:
                break
            i_max[i] = j
            num_cndls += 1
            if i < j_min[j]:
                j_min[j] = i
                
    cdef double[::1] p_ji = np.empty(num_cndls, dtype=np.double)
    cdef double[::1] p_j  = np.empty(num_sizes, dtype=np.double)
    
    cdef int i_num_cndls, i_idx, offset
    cdef double uniform_prob
    
    for i in range(num_sizes):
        i_num_cndls = i_max[i] - i + 1
        i_idx = cndl_idxs[i]
        uniform_prob = 1. / i_num_cndls
        
        for offset in range(i_num_cndls):
            p_ji[i_idx + offset] = uniform_prob
    
    for j in prange(num_sizes, schedule='dynamic', nogil=True):
        update_p_j(j, j_min, cndl_idxs, p_i, p_ji, p_j)
    
    cdef double p_j_result = 0.
    cdef double p_ji_result = 0.
    cdef double last_MI
    
    for j in prange(num_sizes, nogil=True):
        p_j_result += calc_p_j(j, p_j)
        
    for i in prange(num_sizes, schedule='dynamic', nogil=True):
        p_ji_result += calc_p_ji(i, i_max, cndl_idxs, p_ji, p_i)
        
    last_MI = p_j_result + p_ji_result
        
    cdef double this_MI
    cdef double max_L = 0.
    cdef long itr
    
    for itr in range(max_itr):
        
        for i in prange(num_sizes, schedule='dynamic', nogil=True):
            update_p_ji(i, i_max, cndl_idxs, p_ji, p_j)  
            
        for j in prange(num_sizes, schedule='dynamic', nogil=True):
            update_p_j(j, j_min, cndl_idxs, p_i, p_ji, p_j)
        
        if itr % 1 == 0:
            p_j_result = 0.
            p_ji_result = 0.
        
            for j in prange(num_sizes, nogil=True):
                p_j_result += calc_p_j(j, p_j)
        
            for i in prange(num_sizes, schedule='dynamic', nogil=True):
                p_ji_result += calc_p_ji(i, i_max, cndl_idxs, p_ji, p_i)
        
            this_MI = p_j_result + p_ji_result
        
            if this_MI <= last_MI and last_MI - this_MI < eps:    
                for j in prange(num_sizes, schedule='dynamic', nogil=True):
                    max_L += calc_max_L(j, j_min, cndl_idxs, p_ji)
                
                return (this_MI, max_L, itr)
            
            last_MI = this_MI
            
            
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def BlahutArimoto_return_vuln(data, double c, vuln_list, long max_itr = 40, double eps = 1e-4):
    data_sorted = data.groupby(1, as_index=False).agg({2:"sum"})
    
    cdef double total_views = 1. / float(data_sorted[2].sum())
    data_sorted[3] = data_sorted[2] * total_views
    cdef int num_sizes = data_sorted.shape[0]
    
    cdef long[::1] s = data_sorted[1].to_numpy()
    cdef double[::1] p_i = data_sorted[3].to_numpy()
    
    cdef int[::1] i_max = np.empty(num_sizes, dtype=np.intc)
    cdef long[::1] j_min = np.full(num_sizes, num_sizes)
    
    cdef int[::1] cndl_idxs = np.empty(num_sizes, dtype=np.intc)
    
    cdef int i, j
    cdef double max_size
    cdef int num_cndls = 0
    
    for i in range(num_sizes):        
        max_size = float(s[i]) * c
        
        cndl_idxs[i] = num_cndls
        
        for j in range(i,num_sizes):
            if float(s[j]) > max_size:
                break
            i_max[i] = j
            num_cndls += 1
            if i < j_min[j]:
                j_min[j] = i
                
    cdef double[::1] p_ji = np.empty(num_cndls, dtype=np.double)
    cdef double[::1] p_j  = np.empty(num_sizes, dtype=np.double)
    
    cdef int i_num_cndls, i_idx, offset
    cdef double uniform_prob
    
    for i in range(num_sizes):
        i_num_cndls = i_max[i] - i + 1
        i_idx = cndl_idxs[i]
        uniform_prob = 1. / i_num_cndls
        
        for offset in range(i_num_cndls):
            p_ji[i_idx + offset] = uniform_prob
    
    for j in prange(num_sizes, schedule='dynamic', nogil=True):
        update_p_j(j, j_min, cndl_idxs, p_i, p_ji, p_j)
    
    cdef double p_j_result = 0.
    cdef double p_ji_result = 0.
    cdef double last_MI
    
    for j in prange(num_sizes, nogil=True):
        p_j_result += calc_p_j(j, p_j)
        
    for i in prange(num_sizes, schedule='dynamic', nogil=True):
        p_ji_result += calc_p_ji(i, i_max, cndl_idxs, p_ji, p_i)
        
    last_MI = p_j_result + p_ji_result
        
    cdef double this_MI
    cdef double max_L = 0.
    cdef long itr
    
    for itr in range(max_itr):
        
        for i in prange(num_sizes, schedule='dynamic', nogil=True):
            update_p_ji(i, i_max, cndl_idxs, p_ji, p_j)  
            
        for j in prange(num_sizes, schedule='dynamic', nogil=True):
            update_p_j(j, j_min, cndl_idxs, p_i, p_ji, p_j)
        
        if itr % 10 == 0:
            p_j_result = 0.
            p_ji_result = 0.
        
            for j in prange(num_sizes, nogil=True):
                p_j_result += calc_p_j(j, p_j)
        
            for i in prange(num_sizes, schedule='dynamic', nogil=True):
                p_ji_result += calc_p_ji(i, i_max, cndl_idxs, p_ji, p_i)
        
            this_MI = p_j_result + p_ji_result
        
            if this_MI <= last_MI and last_MI - this_MI < eps:    
                break
            
            last_MI = this_MI
    
    size_to_i = {}
    for i in range(num_sizes):
        size_to_i[s[i]] = i
        
    size_is_vuln = defaultdict(int)
    size_no_vuln = defaultdict(int)
    size_counts = defaultdict(int)
    
    cdef long j_size
    cdef double prob
    
    for row in data.itertuples(index=False):
        i = size_to_i[row[1]]        
        i_num_cndls = i_max[i] - i + 1  
        i_idx = cndl_idxs[i]  
        
        if row[0] in vuln_list:
            for offset in range(i_num_cndls):
                j_size = s[i + offset]
                prob = p_ji[i_idx + offset]
                size_is_vuln[j_size] += prob*row[2]
                size_counts[j_size] += prob*row[2]
        else:
            for offset in range(i_num_cndls):
                j_size = s[i + offset]
                prob = p_ji[i_idx + offset]
                size_no_vuln[j_size] += prob*row[2]
                size_counts[j_size] += prob*row[2]
        
    return size_is_vuln, size_no_vuln, size_counts